# Clean and Merge Data
### EOC Datasets
We have three datasets from the EOC scores testing standard (2011-2013)

### Report Card Datasets
We have three datasets from other various testing standard (2014-2024)

### School Geospatial Data
Lat/Long data for each school in King County

In [84]:
# import packages
import pandas as pd

## EOC Data
First merge and clean the EOC Datasets to finish with a csv of schools and their quality scores

In [85]:
# read in csv files
eoc_2011 = pd.read_csv('Original-Data/Scores/EOC Scores by School 2011.csv')
eoc_2012 = pd.read_csv('Original-Data/Scores/EOC Scores by School 2012.csv')
eoc_2013 = pd.read_csv('Original-Data/Scores/EOC Scores by School 2013.csv')

# combine dataframes
eoc = pd.concat([eoc_2011, eoc_2012, eoc_2013], ignore_index=True)

# list of columns to keep
columns_to_keep = ['SchoolYear', 'ESD', 'County', 'District', 'School', 
                   'Algebra1PercentMetStandardIncludingPrevPass', 
                   'GeometryPercentMetStandardIncludingPrevPass', 
                   'BiologyPercentMetStandardIncludingPrevPass']

# filter the DataFrame to keep only the specified columns
eoc_filtered = eoc.filter(columns_to_keep)

# filter for only King County
eoc_king = eoc_filtered[eoc_filtered['County'] == 'King']

# group by school
eoc_king_grouped = eoc_king.groupby(['SchoolYear', 'School']).agg({
    'Algebra1PercentMetStandardIncludingPrevPass': 'mean',
    'GeometryPercentMetStandardIncludingPrevPass': 'mean',
    'BiologyPercentMetStandardIncludingPrevPass': 'mean',
}).reset_index()

# save as csv
eoc_king_grouped.to_csv('Cleaned-Data/EOC/eoc_king_grouped.csv', index=False)

Now create a quality score for each school based on the three EOC scores

In [128]:
# function to calculate the quality score
def quality_score(row):
    algebra = row['Algebra1PercentMetStandardIncludingPrevPass']
    geometry = row['GeometryPercentMetStandardIncludingPrevPass']
    biology = row['BiologyPercentMetStandardIncludingPrevPass']
    
    # Create list of non-NaN values
    valid_scores = [score for score in [algebra, geometry, biology] if pd.notna(score)]

    if len(valid_scores) == 0:
        return 0
    else:
        return sum(valid_scores) / len(valid_scores) / 100

# apply the quality score to the dataframe
eoc_king_grouped['QualityScore'] = eoc_king_grouped.apply(quality_score, axis=1)

print(eoc_king_grouped.head())

  SchoolYear                                  School  \
0    2010-11  Academy of Citizenship and Empowerment   
1    2010-11                Aki Kurose Middle School   
2    2010-11           Albert Einstein Middle School   
3    2010-11                Arts & Academics Academy   
4    2010-11         Auburn Mountainview High School   

   Algebra1PercentMetStandardIncludingPrevPass  \
0                                        34.95   
1                                          NaN   
2                                          NaN   
3                                        21.50   
4                                        36.80   

   GeometryPercentMetStandardIncludingPrevPass  \
0                                        47.83   
1                                          NaN   
2                                          NaN   
3                                        38.00   
4                                        79.20   

   BiologyPercentMetStandardIncludingPrevPass  QualityScore  

## Report Card Data
Next merge and clean the Report Card Datasets to finish with a csv of schools and their quality scores


In [119]:
# read in csv files
rc_2014_22 = pd.read_csv('Original-Data/Scores/Report Card Data 2014-22.csv', low_memory=False)
rc_2022_23 = pd.read_csv('Original-Data/Scores/Report Card Data 2022-23.csv', low_memory=False)
rc_2023_24 = pd.read_csv('Original-Data/Scores/Report Card Data 2023-24.csv', low_memory=False)

# combine dataframes
rc = pd.concat([rc_2014_22, rc_2022_23, rc_2023_24], ignore_index=True)


In [121]:
# rename columns and handle duplicates
rc = rc.rename(columns={'ESDOrganizationID': 'ESD', 
                       'DistricName': 'District', 
                       'Percent Foundational Grade-Level Knowledge And Above': 'PercentMetStandard'})

# Combine duplicate PercentMetStandard columns if they exist
if rc.columns.duplicated().any():
    percent_cols = [i for i, col in enumerate(rc.columns) if col == 'PercentMetStandard']
    rc['PercentMetStandard'] = rc.iloc[:, percent_cols[0]].combine_first(rc.iloc[:, percent_cols[1]])
    rc = rc.loc[:, ~rc.columns.duplicated(keep='first')]

# filter for only King County and All Students
rc_king = rc[rc['County'] == 'King']
rc_king = rc_king[rc_king['StudentGroupType'] == 'All']
rc_king = rc_king[rc_king['GradeLevel'] == 'All Grades']

# list of columns to keep
columns_to_keep = ['SchoolYear', 'ESD', 'District', 
                   'SchoolCode', 'SchoolName','PercentMetStandard']

# filter the DataFrame to keep only the specified columns
rc_king_filtered = rc_king.filter(columns_to_keep)

# Replace non-numeric values
rc_king_filtered['PercentMetStandard'] = rc_king_filtered['PercentMetStandard'].str.replace('No Students', '0')
rc_king_filtered['PercentMetStandard'] = rc_king_filtered['PercentMetStandard'].str.replace('>', '').str.replace('<', '')
rc_king_filtered['PercentMetStandard'] = rc_king_filtered['PercentMetStandard'].str.replace('Suppressed: N10', '5').str.replace('N10 (Count Protected)', '5').str.replace('N10', '5')

# # convert percent met standard to float
rc_king_filtered['PercentMetStandard'] = rc_king_filtered['PercentMetStandard'].str.replace('%', '').astype(float)

# group by school
rc_king_grouped = rc_king_filtered.groupby(['SchoolYear', 'SchoolCode']).agg({
    'PercentMetStandard': 'mean',
}).reset_index()

rc_king_grouped.rename(columns={'PercentMetStandard': 'QualityScore'}, inplace=True)
rc_king_grouped['QualityScore'] = rc_king_grouped['QualityScore'] / 100

print(rc_king_grouped.head())

  SchoolYear  SchoolCode  QualityScore
0    2014-15      1502.0         0.206
1    2014-15      1523.0         0.050
2    2014-15      1534.0         0.050
3    2014-15      1539.0         0.348
4    2014-15      1540.0         0.124


In [95]:
# save csvs
rc_king_grouped.to_csv('Cleaned-Data/Report-Card/rc_king_grouped.csv', index=False)
eoc_king_grouped.to_csv('Cleaned-Data/EOC/eoc_king_grouped.csv', index=False)

## Add Geospatial Data
We will merge the geospatial data with the EOC and Report Card data to finish with a csv of schools and their quality scores and lat/long data

In [129]:
# read in csv files
school_geospatial = pd.read_csv('Original-Data/School Geospatial/Washington State Public Schools 2024.csv')
rc = rc_king_grouped.copy()
eoc = eoc_king_grouped.copy()

# make rc and eoc dataframes compatible
eoc.drop(columns=['Algebra1PercentMetStandardIncludingPrevPass', 'GeometryPercentMetStandardIncludingPrevPass', 'BiologyPercentMetStandardIncludingPrevPass'], inplace=True)
eoc.rename(columns={'School': 'SchoolName'}, inplace=True)
rc['SchoolCode'] = rc['SchoolCode'].astype(int)

print(eoc.head())
print(rc.head())

  SchoolYear                              SchoolName  QualityScore
0    2010-11  Academy of Citizenship and Empowerment        0.4139
1    2010-11                Aki Kurose Middle School        0.0000
2    2010-11           Albert Einstein Middle School        0.0000
3    2010-11                Arts & Academics Academy        0.2975
4    2010-11         Auburn Mountainview High School        0.5800
  SchoolYear  SchoolCode  QualityScore
0    2014-15        1502         0.206
1    2014-15        1523         0.050
2    2014-15        1534         0.050
3    2014-15        1539         0.348
4    2014-15        1540         0.124


Now merge the dataframes with the geospatial data

In [136]:
# prepare the geospatial data
school_geospatial = school_geospatial[school_geospatial['County'] == 'King']  # Filter for King County

# Merge EOC data with geospatial using school name
merged_eoc = pd.merge(
    eoc,
    school_geospatial[['School', 'School Code', 'GeoCoded_Y', 'GeoCoded_X']],
    left_on='SchoolName',
    right_on='School',
    how='left'
)

# Merge RC data with geospatial using school code
merged_rc = pd.merge(
    rc,
    school_geospatial[['School', 'School Code', 'GeoCoded_Y', 'GeoCoded_X']],
    left_on='SchoolCode',
    right_on='School Code',
    how='left'
)

# concat the two dataframes
merged = pd.concat([merged_eoc, merged_rc], ignore_index=True)

# drop rows where GeoCoded_X or GeoCoded_Y are null
merged.dropna(subset=['GeoCoded_X', 'GeoCoded_Y'], inplace=True)

# drop unnecessary columns
merged.drop(columns=['SchoolName', 'School Code', 'SchoolCode'], inplace=True)

print(merged.head())

     SchoolYear  QualityScore                          School  GeoCoded_Y  \
1       2010-11           0.0        Aki Kurose Middle School   47.546628   
2       2010-11           0.0   Albert Einstein Middle School   47.769758   
11      2010-11           0.0              Big Picture School   47.437033   
13      2010-11           0.0    Broadview-Thomson K-8 School   47.724657   
16      2010-11           0.0                            CLIP   47.652353   
...         ...           ...                             ...         ...   
1068    2012-13           0.0     Wing Luke Elementary School   47.530546   
2048    2015-16           0.0    Summit Public School: Sierra   47.597854   
2575    2017-18           0.0         Regional Justice Center   47.384679   
2587    2017-18           0.0           Satellite High School   47.437805   
4020    2020-21           0.0  Renton Technical College - You   47.489168   

      GeoCoded_X  
1    -122.282550  
2    -122.362938  
11   -122.328097  

In [131]:
# save final csv
merged.to_csv('Cleaned-Data/merged.csv', index=False)